In [ ]:
import numpy as np
import sys
# option to import from github folder
#sys.path.insert(0, '/media/carsen/DATA2/Github/suite2p')
sys.path.insert(0, 'C:/Users/carse/github/suite2p')

import imp
from suite2p import run_s2p
from suite2p import utils
from skimage.external.tifffile import imread
from ScanImageTiffReader import ScanImageTiffReader

fs = ['/media/carsen/DATA2/grive/suite2p_test/example.tif']

fs = ['/media/carsen/DATA2/grive/suite2p_test/2016-07-27_8_M160704_FR096_2P_001_001.tif']

fs = ['/media/carsen/DATA1/TIFFS/SS05/20160517_SS05_02.tif']

#fs = ['C:/Users/carse/Downloads/neurodroid/20180919_BG0039_sensor0_0000_corr.tif']
fs = ['/media/carsen/DATA1/TIFFS/neurodroid/20180919_BG0039_sensor0_0000_corr.tif']

#for f in fs:
    
#    print(tif.shape())
#fs = ['C:/Users/carse/Downloads/data_1p.tif']

In [ ]:
data = imread(fs[0])
#im=tif.data()
print(data.shape)

In [ ]:
ops={'smooth_sigma': 5}
ops['spatial_hp'] = 50
ops['pre_smooth'] = 0
ops['Ly'] = data.shape[1]
ops['Lx'] = data.shape[2]
ops['1Preg'] = 1
ops['do_phasecorr'] = True
ops['nonrigid'] = False
ops['num_workers'] = 4
ops['maxregshift'] = 0.15
ops['subpixel'] = 10


In [ ]:
ops={'smooth_sigma': 1}
ops['spatial_hp'] = 2
ops['pre_smooth'] = 8
ops['Ly'] = data.shape[1]
ops['Lx'] = data.shape[2]
ops['1Preg'] = 1
ops['do_phasecorr'] = False#True
ops['nonrigid'] = False
ops['num_workers'] = 4
ops['maxregshift'] = 0.15
ops['subpixel'] = 10


In [ ]:
from suite2p import register
import imp
imp.reload(register)


refImg=register.pick_init_init(ops, data[::6,:,:])
refImg =register.refine_init_init(ops, data[::6,:,:], refImg)

plt.imshow(refImg)

In [ ]:
imp.reload(register)


maskMul, maskOffset, cfRefImg = register.prepare_masks(refImg.copy(), ops)

plt.figure(figsize=(10,10))
plt.imshow(np.real(fft.ifft2(np.conj(cfRefImg))))
#plt.imshow(fhg)

In [ ]:
plt.imshow(maskMul)

In [ ]:
dwrite, ymax, xmax, cmax, yxnr = register.phasecorr(data[:200,:,:], [maskMul, maskOffset, cfRefImg], ops)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(ymax[:])
plt.plot(xmax[:])
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots(figsize=(10,10))

#line, = ax.plot([], [], lw=2)
im = ax.imshow(dwrite[0,:,:],cmap='gray')

# initialization function: plot the background of each frame
def init():
    im.set_data(dwrite[0,:,:])#,cmap='gray')
    return (im,)
# animation function. This is called sequentially
def animate(i):
    im.set_data(dwrite[i,:,:])#,cmap='gray')
    return (im,)
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=100, blit=True)
HTML(anim.to_html5_video())

In [ ]:
plt.figure(figsize=(20,10))
ix=186
vm=-40
vx=40
plt.subplot(1,2,1)
plt.imshow(data[ix,:,:]-data[ix+1,:,:],vmin=vm,vmax=vx)
plt.subplot(1,2,2)
plt.imshow(dout[ix,:,:]-dout[ix+1,:,:],vmin=vm,vmax=vx)

In [ ]:
dout=data

In [ ]:
from skimage.external.tifffile import TiffFile

#fs = ['/home/carsen/TIFFS/TSeries-20170508-D43-001_Cycle00001_Ch2_000001.ome.tif']
fs = ['/home/carsen/TIFFS/MP032/2017-08-01_6_M170714_MP032_2P_001_001.tif'];

import time

tic = time.time()
tif = TiffFile(fs[0])
print(time.time()-tic)

print(tif[0].asarray())

In [ ]:
1800/12 * 5

In [ ]:
F = np.load('/home/carsen/Downloads/suite2p/plane0/F.npy')
F.shape

In [ ]:
import scipy.io

Fall = scipy.io.loadmat('/home/carsen/Downloads/suite2p/plane0/Fall.mat',squeeze_me=True)

In [ ]:
Fall['F'].shape